### Sentiment Analysis

We want to do sentiment analysis by using [VaderSentiment](https://github.com/cjhutto/vaderSentiment) ML framework not supported as an MLflow Flavor. The goal of sentiment analysis is to "gauge the attitude, sentiments, evaluations, attitudes and emotions of a speaker/writer based on the computational treatment of subjectivity in a text."

VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media.

VADER has a lot of advantages over traditional methods of Sentiment Analysis, including:

 * It works exceedingly well on social media type text, yet readily generalizes to multiple domains
 * It doesn’t require any training data but is constructed from a generalizable, valence-based, human-curated gold standard sentiment lexicon
 * It is fast enough to be used online with streaming data, and
 * It does not severely suffer from a speed-performance tradeoff.
 
 <table>
  <tr><td>
    <img src="https://github.com/dmatrix/olt-mlflow/raw/master/models/images/sentiment_analysis.jpg"
         alt="Sentiment Analysis with Vader" height="400 width="600">
  </td></tr>
</table>

[image source](https://medium.com/analytics-vidhya/sentiment-analysis-with-vader-label-the-unlabeled-data-8dd785225166)

Need to install the package

In [1]:
%pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


### VaderSentiment Python Package

You can read the orignal paper by authors [here](http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf).

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
from ray import serve

Define some input texts we going to analyse 

In [3]:
INPUT_TEXTS = [{'text': "This is a bad ass movie. You got to see it! :-)"},
               {'text': "Ricky Gervais is smart, witty, and creative!!!!!! :D"},
               {'text': "LOL, this guy fell off a chair while sleeping and snoring in a meeting"},
               {'text': "Men shoots himself while trying to steal a dog, OMG"},
               {'text': "Ray and Ray Serve just rocks. Love the way you easily define Ray Actors. Simple APIs, and they work!"},
               {'text': "Yay!! Another good phone interview. I nailed it!!"},
               {'text': "This is INSANE! I can't believe it. How could you do such a horrible thing?"}]

### Start the Ray Serve. It automatically starts Ray processes on the local host

In [4]:
serve.start()

2021-10-31 18:15:17,865	INFO services.py:1252 -- View the Ray dashboard at http://127.0.0.1:8266
(pid=98415) 2021-10-31 18:15:19,744	INFO checkpoint_path.py:15 -- Using RayInternalKVStore for controller checkpoint and recovery.
(pid=98415) 2021-10-31 18:15:19,747	INFO http_state.py:78 -- Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:HwBUTs:SERVE_PROXY_ACTOR-node:192.168.86.49-0' on node 'node:192.168.86.49-0' listening on '127.0.0.1:8000'
2021-10-31 18:15:19,989	INFO api.py:455 -- Started Serve instance in namespace 'serve'.


(pid=98420) INFO:     Started server process [98420]


### Define and a SocialMediaAnalyserModel

In [5]:
@serve.deployment(route_prefix="/sentiments")
class SocialMediaAnalyserModel(object):
    def __init__(self):
      """
      Constructor for our Sentiment Analyser
      """
      # Initialize an instance of vader analyser
      self._analyser = SentimentIntensityAnalyzer()

    async def __call__(self, starlette_request):
        payload = await starlette_request.json()
        text = payload['text']
        print("Worker: received starlette request with sentimet text", text)
        scores = self._score(text)
        print(f"<{text}> --> {str(scores)}>")
        
    def _score(self, text):
        """
        Private function to analyse the scores. It invokes model's 
        param: text to analyse
        return: sentiment analyses scores
        """
        scores = self._analyser.polarity_scores(text)
        return scores

## Deploy the model

In [6]:
SocialMediaAnalyserModel.deploy()

2021-10-31 18:15:29,379	INFO api.py:243 -- Updating deployment 'SocialMediaAnalyserModel'. component=serve deployment=SocialMediaAnalyserModel
(pid=98415) 2021-10-31 18:15:29,392	INFO backend_state.py:896 -- Adding 1 replicas to deployment 'SocialMediaAnalyserModel'. component=serve deployment=SocialMediaAnalyserModel
2021-10-31 18:15:29,714	INFO api.py:251 -- Deployment 'SocialMediaAnalyserModel' is ready at `http://127.0.0.1:8000/sentiments`. component=serve deployment=SocialMediaAnalyserModel


We can now send HTTP requests to our route `route_prefix=/sentiments` at the default port 8000

In [7]:
import requests  # for making web requests
for sentiment in INPUT_TEXTS:
    response = requests.get(
    "http://localhost:8000/sentiments", json=sentiment)
    print(response.text)








(SocialMediaAnalyserModel pid=98413) Worker: received starlette request with sentimet text This is a bad ass movie. You got to see it! :-)
(SocialMediaAnalyserModel pid=98413) <This is a bad ass movie. You got to see it! :-)> --> {'neg': 0.0, 'neu': 0.542, 'pos': 0.458, 'compound': 0.7644}>
(SocialMediaAnalyserModel pid=98413) Worker: received starlette request with sentimet text Ricky Gervais is smart, witty, and creative!!!!!! :D
(SocialMediaAnalyserModel pid=98413) <Ricky Gervais is smart, witty, and creative!!!!!! :D> --> {'neg': 0.0, 'neu': 0.316, 'pos': 0.684, 'compound': 0.8957}>
(SocialMediaAnalyserModel pid=98413) Worker: received starlette request with sentimet text LOL, this guy fell off a chair while sleeping and snoring in a meeting
(SocialMediaAnalyserModel pid=98413) <LOL, this guy fell off a chair while sleeping and snoring in a meeting> --> {'neg': 0.0, 'neu': 0.786, 'pos': 0.214, 'compound': 0.5473}>
(SocialMediaAnalyserModel pid=98413) Worker: received starlet

## Cleanup

In [8]:
deployments = serve.list_deployments()
print(f'deployments: {deployments}')

deployments: {'SocialMediaAnalyserModel': Deployment(name=SocialMediaAnalyserModel,version=None,route_prefix=/sentiments)}


In [9]:
serve.shutdown()

(pid=98415) 2021-10-31 18:17:30,293	INFO backend_state.py:914 -- Removing 1 replicas from deployment 'SocialMediaAnalyserModel'. component=serve deployment=SocialMediaAnalyserModel


## Exercise - Try Adding more examples

Here are some things you can try:

1. Add more neutral, negative and positive tests.
2. "Automate" the steps we did:
    * Wrap the training in a function.
    * Run the whole sequence of retraining every minute for a few minutes.